# Background

We aim to compute the following quantity:
$$
\mathbb{P}(\Omega^{\text{vis}}) 
  = \sum_{k=2}^{n} \mathbb{P}(\Omega^{\text{vis}}_k)
$$
whereby
\begin{equation}
\mathbb{P}(\Omega^{\text{vis}}_k) = p^{\neg \rho}_{n,k} \frac{\rho}{k-1+\rho} q_{k+1,(0,0)}
\end{equation}

In order to do this, we need to compute a series of **auxiliary variables**:
\begin{equation}
p^{\neg \rho}_{n_1,n_2} = \prod_{n_2 < k \leq n_1} \frac{k-1}{k-1+\rho}
\end{equation}

The probabilities $(q_{k,{i,j}})_{k \in \mathbb{N}, i,j \in \lbrace 0,1,2 \rbrace}$ satisfy the following recursive linear system:
\begin{align}
q_{k,(0,0)} &= \frac{(k-2) ( q_{k-1,(1,0)} + q_{k-1,(0,1)}) + \binom{k-2}{2} q_{k-1,(0,0)} }{\binom{k}{2} + \frac{k}{2} \rho} \\
q_{k,(0,1)} &= \frac{\frac{1}{2} \theta_2 \, q_{k,(0,2)} + (k-2) \, q_{k-1,(1,1)} + \binom{k-1}{2} \, q_{k-1,(0,1)} }{\binom{k}{2} + \frac{k}{2} \rho + \frac{1}{2}\theta_2} \\
q_{k,(1,0)} &= \frac{\frac{1}{2} \theta_1 \, q_{k,(2,0)} + (k-2) \, q_{k-1,(1,1)} + \binom{k-1}{2} \, q_{k-1,(1,0)} }{\binom{k}{2} + \frac{k}{2} \rho + \frac{1}{2}\theta_1} \\
q_{k,(0,2)} &= \frac{(k-2) q_{k-1,(1,2)} + \binom{k-1}{2} q_{k-1,(0,2)} }{\binom{k}{2} + \frac{k}{2} \rho} \\
q_{k,(2,0)} &= \frac{(k-2) q_{k-1,(2,1)} + \binom{k-1}{2} q_{k-1,(2,1)} }{\binom{k}{2} + \frac{k}{2} \rho} \\q_{k,(1,1)} &= \frac{\frac{1}{2} \theta_1 \, q_{k,(2,1)} + \frac{1}{2} \theta_2 q_{k,(1,2)} + \left( \binom{k}{2} - 1 \right) \, q_{k-1,(1,1)} }{\binom{k}{2} + \frac{k}{2} \rho + \frac{1}{2} ( \theta_1 + \theta_2 ) } \\
q_{k,(1,2)} &= \frac{\frac{1}{2} \theta_1 q_{k,(2,2)} + \binom{k-1}{2} q_{k-1,(1,2)}}{\binom{k}{2} + \frac{k}{2} \rho + \frac{1}{2}\theta_1}\\
q_{k,(2,1)} &= \frac{\frac{1}{2} \theta_2 q_{k,(2,2)} + \binom{k-1}{2} q_{k-1,(2,1)}}{\binom{k}{2} + \frac{k}{2} \rho + \frac{1}{2}\theta_2}
\end{align}

with boundary conditions:
\begin{align}
q_{k,(0,0)} &= 0 \text{ if } k<4 \\
q_{k,(0,1)} &= 0 \text{ if } k<3 \\
q_{k,(1,0)} &= 0 \text{ if } k<3 \\
q_{k,(0,2)} &= 0 \text{ if } k<3 \\
q_{k,(2,0)} &= 0 \text{ if } k<3 \\
q_{k,(1,1)} &= 0 \text{ if } k<2 \\
q_{k,(1,2)} &= 0 \text{ if } k<2 \\
q_{k,(2,1)} &= 0 \text{ if } k<2 \\
q_{k,(2,2)} &= p^{\neg \rho}_{k,1} = \prod_{i = 2}^k \frac{i-1}{i-1+\rho}
\end{align}

The solution of this system will be a series of rational functions of integers $n \geq k > 1$ and non-negative reals $\rho, \theta_1, \theta_2$.

In [82]:
def p_neg(n1,n2,rho):
    'The probability that a set of n1 lineages will coalesce to n2 lineages before experiencing recurison'
    assert n1 >= n2
    assert n2 > 0

    if isinstance(rho,int):
        rho = float(rho)
    if isinstance(rho,float):
        assert rho >= 0
    
    return reduce( lambda x,y: x*y, [(k - 1)/(k-1+rho) for k in range(n1,n2,-1)], 1)

In [101]:
def q_recursive(k,i,j,theta1,theta2,rho):
    'compute q_{k,(i,j)} using tail-recursion; theta1,theta2, and rho should be non-negative reals.'
    
    if isinstance(rho,int):
        rho = float(rho)
    if isinstance(theta1,int):
        theta1 = float(theta1)
    if isinstance(theta2,int):
        theta2 = float(theta2)
    
    if   (i,j) == (0,0):
        if k < 4:
            return 0
        else:
            a = q_recursive(k-1, 1, 0, theta1,theta2,rho)
            b = q_recursive(k-1, 0, 1, theta1,theta2,rho)
            c = q_recursive(k-1, 0, 0, theta1,theta2,rho)
            result = ( (k - 2)*(a+b) + choose2(k-2)*c ) / (chose2(k) + k*rho/2)
            return result
            
    elif (i,j) == (0,1):
        if k < 3:
            return 0
        else:
            a = q_recursive(k  , 0, 2, theta1,theta2,rho)
            b = q_recursive(k-1, 1, 1, theta1,theta2,rho)
            c = q_recursive(k-1, 0, 1, theta1,theta2,rho)
            result = ( a*theta2/2 + (k - 2)*b + choose2(k-1)*c ) / ( chose2(k) + (k*rho + theta2)/2 )
            return result

    elif (i,j) == (1,0):
        if k < 3:
            return 0
        else:
            a = q_recursive(k  , 2, 0, theta1,theta2,rho)
            b = q_recursive(k-1, 1, 1, theta1,theta2,rho)
            c = q_recursive(k-1, 1, 0, theta1,theta2,rho)
            result = ( a*theta1/2 + (k - 2)*b + choose2(k-1)*c ) / ( chose2(k) + (k*rho + theta1)/2 )
            return result

    elif (i,j) == (0,2):
        if k < 3:
            return 0
        else:
            a = q_recursive(k-1, 1, 2, theta1,theta2,rho)
            b = q_recursive(k-1, 0, 2, theta1,theta2,rho)
            result = ( (k - 2)*a + choose2(k-1)*b ) / ( chose2(k) + k*rho/2 )
            return result
    
    elif (i,j) == (2,0):
        if k < 3:
            return 0
        else:
            a = q_recursive(k-1, 2, 1, theta1,theta2,rho)
            b = q_recursive(k-1, 2, 0, theta1,theta2,rho)
            result = ( (k - 2)*a + choose2(k-1)*b ) / ( chose2(k) + k*rho/2 )
            return result
    
    elif (i,j) == (1,1):
        if k < 2:
            return 0
        else:
            a = q_recursive(k  , 2, 1, theta1,theta2,rho)
            b = q_recursive(k  , 1, 2, theta1,theta2,rho)
            c = q_recursive(k-1, 1, 1, theta1,theta2,rho)
            result = ( (theta1*a + theta2*b)/2 + (choose2(k) - 1)*c ) / ( chose2(k) + (k*rho + theta1 + theta2)/2 )
            return result
    
    elif (i,j) == (1,2):
        if k < 2:
            return 0
        else:
            a = q_recursive(k  , 2, 2, theta1,theta2,rho)
            b = q_recursive(k-1, 1, 2, theta1,theta2,rho)
            result = ( a*theta1/2 + choose2(k-1)*b ) / ( chose2(k) + (k*rho + theta1)/2 )
            return result
    
    elif (i,j) == (2,1):
        if k < 2:
            return 0
        else:
            a = q_recursive(k  , 2, 2, theta1,theta2,rho)
            b = q_recursive(k-1, 2, 1, theta1,theta2,rho)
            result = ( a*theta2/2 + choose2(k-1)*b ) / ( chose2(k) + (k*rho + theta2)/2 )
            return result
    
    elif (i,j) == (2,2):
        result = p_neg(k,1,rho)
        return result
    
    else:
        raise ValueError('0 <= i,j <= 2 must hold; arguments here: i = %i, j=%i'%(i,j))

def choose2(k):
    return (k*(k-1))/2

In [70]:
q_recursive(50,0,0,1e10,1e10,0.0)

0.933605437401415

### Symbolic computation
The above implmentation also supports symbolic computation. We may use this to get exact formulas expressing $q_{k,(0,0)}$ as a rational function in the indeterminates $\theta_1, \theta_2$, and $\rho$ (the below does just this using sympy):

In [139]:
import sympy

In [140]:
k = 4
t, t1,t2,r = sympy.symbols(r'\theta \theta_1 \theta_2 \rho')
f = q_recursive(k,0,0,t1,t2,r) # general formula
g = f.subs(r,0) # f with \rho = 0
h = q_recursive(k,0,0,t/2,t/2,r) # f with theta1 = theta2 = theta/2

In [141]:
print f,'\n'
print g,'\n'
print h,'\n'

(2*(\theta_1*\theta_2/(4*(\rho + 1)*(3*\rho/2 + 3)*(\rho + \theta_1/2 + 1)) + (\theta_1*\theta_2/(4*(\rho + 1)*(\rho + \theta_2/2 + 1)) + \theta_1*\theta_2/(4*(\rho + 1)*(\rho + \theta_1/2 + 1)))/(\rho + \theta_1/2 + \theta_2/2 + 1))/(3*\rho/2 + \theta_2/2 + 3) + 2*(\theta_1*\theta_2/(4*(\rho + 1)*(3*\rho/2 + 3)*(\rho + \theta_2/2 + 1)) + (\theta_1*\theta_2/(4*(\rho + 1)*(\rho + \theta_2/2 + 1)) + \theta_1*\theta_2/(4*(\rho + 1)*(\rho + \theta_1/2 + 1)))/(\rho + \theta_1/2 + \theta_2/2 + 1))/(3*\rho/2 + \theta_1/2 + 3))/(2*\rho + 6) 

(\theta_1*\theta_2/(12*(\theta_1/2 + 1)) + (\theta_1*\theta_2/(4*(\theta_2/2 + 1)) + \theta_1*\theta_2/(4*(\theta_1/2 + 1)))/(\theta_1/2 + \theta_2/2 + 1))/(3*(\theta_2/2 + 3)) + (\theta_1*\theta_2/(12*(\theta_2/2 + 1)) + (\theta_1*\theta_2/(4*(\theta_2/2 + 1)) + \theta_1*\theta_2/(4*(\theta_1/2 + 1)))/(\theta_1/2 + \theta_2/2 + 1))/(3*(\theta_1/2 + 3)) 

4*(\theta**2/(8*(\rho + 1)*(\rho + \theta/4 + 1)*(\rho + \theta/2 + 1)) + \theta**2/(16*(\rho + 1)*(3

### We may also print these expressions in latex:

In [142]:
print sympy.latex(f),'\n'
print sympy.latex(f.simplify()),'\n\n'
print sympy.latex(g),'\n'
print sympy.latex(g.simplify()),'\n\n'
print sympy.latex(h),'\n'
print sympy.latex(h.simplify()),'\n\n'
#print sympy.latex(g)

\frac{1}{2 \rho + 6} \left(\frac{1}{\frac{3 \rho}{2} + \frac{\theta_2}{2} + 3} \left(\frac{\theta_1 \theta_2}{2 \left(\rho + 1\right) \left(\frac{3 \rho}{2} + 3\right) \left(\rho + \frac{\theta_1}{2} + 1\right)} + \frac{1}{\rho + \frac{\theta_1}{2} + \frac{\theta_2}{2} + 1} \left(\frac{\theta_1 \theta_2}{2 \left(\rho + 1\right) \left(\rho + \frac{\theta_2}{2} + 1\right)} + \frac{\theta_1 \theta_2}{2 \left(\rho + 1\right) \left(\rho + \frac{\theta_1}{2} + 1\right)}\right)\right) + \frac{1}{\frac{3 \rho}{2} + \frac{\theta_1}{2} + 3} \left(\frac{\theta_1 \theta_2}{2 \left(\rho + 1\right) \left(\frac{3 \rho}{2} + 3\right) \left(\rho + \frac{\theta_2}{2} + 1\right)} + \frac{1}{\rho + \frac{\theta_1}{2} + \frac{\theta_2}{2} + 1} \left(\frac{\theta_1 \theta_2}{2 \left(\rho + 1\right) \left(\rho + \frac{\theta_2}{2} + 1\right)} + \frac{\theta_1 \theta_2}{2 \left(\rho + 1\right) \left(\rho + \frac{\theta_1}{2} + 1\right)}\right)\right)\right) 

\frac{2 \theta_1 \theta_2 \left(\left(3 \left(\rho

this latex-source code can then be typeset:

### *f* :
$$\frac{1}{2 \rho + 6} \left(\frac{1}{\frac{3 \rho}{2} + \frac{\theta_2}{2} + 3} \left(\frac{\theta_1 \theta_2}{2 \left(\rho + 1\right) \left(\frac{3 \rho}{2} + 3\right) \left(\rho + \frac{\theta_1}{2} + 1\right)} + \frac{1}{\rho + \frac{\theta_1}{2} + \frac{\theta_2}{2} + 1} \left(\frac{\theta_1 \theta_2}{2 \left(\rho + 1\right) \left(\rho + \frac{\theta_2}{2} + 1\right)} + \frac{\theta_1 \theta_2}{2 \left(\rho + 1\right) \left(\rho + \frac{\theta_1}{2} + 1\right)}\right)\right) + \frac{1}{\frac{3 \rho}{2} + \frac{\theta_1}{2} + 3} \left(\frac{\theta_1 \theta_2}{2 \left(\rho + 1\right) \left(\frac{3 \rho}{2} + 3\right) \left(\rho + \frac{\theta_2}{2} + 1\right)} + \frac{1}{\rho + \frac{\theta_1}{2} + \frac{\theta_2}{2} + 1} \left(\frac{\theta_1 \theta_2}{2 \left(\rho + 1\right) \left(\rho + \frac{\theta_2}{2} + 1\right)} + \frac{\theta_1 \theta_2}{2 \left(\rho + 1\right) \left(\rho + \frac{\theta_1}{2} + 1\right)}\right)\right)\right)$$

### *f* (simplified) :
$$\frac{2 \theta_1 \theta_2 \left(\left(3 \left(\rho + 2\right) \left(4 \rho + \theta_1 + \theta_2 + 4\right) + \left(2 \rho + \theta_1 + 2\right) \left(2 \rho + \theta_1 + \theta_2 + 2\right)\right) \left(3 \rho + \theta_2 + 6\right) + \left(3 \left(\rho + 2\right) \left(4 \rho + \theta_1 + \theta_2 + 4\right) + \left(2 \rho + \theta_2 + 2\right) \left(2 \rho + \theta_1 + \theta_2 + 2\right)\right) \left(3 \rho + \theta_1 + 6\right)\right)}{3 \left(\rho + 1\right) \left(\rho + 2\right) \left(\rho + 3\right) \left(2 \rho + \theta_1 + 2\right) \left(2 \rho + \theta_2 + 2\right) \left(3 \rho + \theta_1 + 6\right) \left(3 \rho + \theta_2 + 6\right) \left(2 \rho + \theta_1 + \theta_2 + 2\right)} $$

### *g* :
$$
\frac{1}{\frac{3 \theta_2}{2} + 9} \left(\frac{\theta_1 \theta_2}{6 \theta_1 + 12} + \frac{1}{\frac{\theta_1}{2} + \frac{\theta_2}{2} + 1} \left(\frac{\theta_1 \theta_2}{2 \theta_2 + 4} + \frac{\theta_1 \theta_2}{2 \theta_1 + 4}\right)\right) + \frac{1}{\frac{3 \theta_1}{2} + 9} \left(\frac{\theta_1 \theta_2}{6 \theta_2 + 12} + \frac{1}{\frac{\theta_1}{2} + \frac{\theta_2}{2} + 1} \left(\frac{\theta_1 \theta_2}{2 \theta_2 + 4} + \frac{\theta_1 \theta_2}{2 \theta_1 + 4}\right)\right)$$

### *g*  (simplified) :
$$
\frac{\theta_1 \theta_2}{9 \left(\theta_1 + 2\right) \left(\theta_1 + 6\right) \left(\theta_2 + 2\right) \left(\theta_2 + 6\right) \left(\theta_1 + \theta_2 + 2\right)} \left(\left(\theta_1 + 6\right) \left(6 \theta_1 + 6 \theta_2 + \left(\theta_2 + 2\right) \left(\theta_1 + \theta_2 + 2\right) + 24\right) + \left(\theta_2 + 6\right) \left(6 \theta_1 + 6 \theta_2 + \left(\theta_1 + 2\right) \left(\theta_1 + \theta_2 + 2\right) + 24\right)\right) $$

### *h* :
$$\frac{1}{\left(2 \rho + 6\right) \left(\frac{3 \rho}{2} + \frac{\theta}{4} + 3\right)} \left(\frac{\theta^{2}}{2 \left(\rho + 1\right) \left(\rho + \frac{\theta}{4} + 1\right) \left(\rho + \frac{\theta}{2} + 1\right)} + \frac{\theta^{2}}{4 \left(\rho + 1\right) \left(\frac{3 \rho}{2} + 3\right) \left(\rho + \frac{\theta}{4} + 1\right)}\right)$$
 ### *h* (simplified) :
 $$
\frac{4 \theta^{2} \left(8 \rho + \theta + 14\right)}{3 \left(\rho + 1\right) \left(\rho + 2\right) \left(\rho + 3\right) \left(2 \rho + \theta + 2\right) \left(4 \rho + \theta + 4\right) \left(6 \rho + \theta + 12\right)} 
$$

## Towards conditionals

We now define the following:

\begin{align}
\mathbb{P} (\Omega_k)
   &=  \frac{\rho}{k-1 + \rho} \frac{k}{k + \rho} \prod_{i=2}^n \frac{i-1}{i - 1 + \rho} \\
   &=  \frac{\rho}{k-1 + \rho} \frac{k}{k + \rho} p^{\neg \rho}_{n,1}
\end{align}
and
\begin{align}
\mathbb{P}(\Omega_{2,\ldots,n})
   &=  \sum_{k = 2}^n \mathbb{P} (\Omega_k) \\
   &=  p^{\neg \rho}_{n,1} \sum_{k = 2}^n \frac{\rho}{k-1 + \rho} \frac{k}{k + \rho}
\end{align}

In [144]:
def P_omega_k(k,n,rho):
    return p_neg(n,1,rho) * ( (rho*k) / ( (k-1+rho) * (k+rho) ) )

def P_omega_2toN(n,rho):
    return p_neg(n,1,rho) * sum([(rho*k) / ( (k-1+rho) * (k+rho) ) for k in xrange(2,n)])

In [68]:
P_omega_k(2,2,1.0)

0.16666666666666666